https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset

In [34]:
from google.colab import output
#Install Keras-tuner on first run.
!pip install keras-tuner
# install the ydata-profiling package
!pip install ydata-profiling
#Mount google drive on first run
from google.colab import drive
drive.mount('/content/drive')
#Clear cell output
output.clear() #Clears output after installing packages. Hide if any issues.

In [104]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from tensorflow import keras
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import keras_tuner as kt
import os

#  Import and read the charity_data.csv.
url = "drive/MyDrive/BootCamp/project_4/Resources/healthcare-dataset-stroke-data.csv"
df = pd.read_csv(url)

#Import python auto EDA
# load the ydata_profiling package
from ydata_profiling import ProfileReport

#drop id column
df = df.drop(columns=['id'])



# Data Cleaning

In [106]:
#Need to deal with NaN BMI
#Dropping bmi as earlier investigation suggests that it is a low correlation to the target feature.
df_clean = df.drop(columns=['bmi'])

In [107]:
df_clean.dtypes

gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
smoking_status        object
stroke                 int64
dtype: object

In [108]:
# Convert categorical data to numeric with `pd.get_dummies`
#Get list of object columns
y = df_clean['stroke'].values
df_clean = df_clean.drop(columns='stroke')
object_columns = df_clean.select_dtypes(include=['object']).columns


#encode
df_clean_onehot = pd.get_dummies(df_clean, columns=object_columns)
df_clean_onehot = df_clean_onehot.astype(float)
df_clean_onehot.head()

,age,hypertension,heart_disease,avg_glucose_level,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0.0,1.0,228.69,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,61.0,0.0,0.0,202.21,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,80.0,0.0,1.0,105.92,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,49.0,0.0,0.0,171.23,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,79.0,1.0,0.0,174.12,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


Prepare datset for ML

In [109]:
# Split our preprocessed data into our features and target arrays
X = df_clean_onehot.values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [110]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [111]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_total = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features_total, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 8)                 168       
                                                                 
 dense_25 (Dense)            (None, 5)                 45        
                                                                 
 dense_26 (Dense)            (None, 1)                 6         
                                                                 
Total params: 219
Trainable params: 219
Non-trainable params: 0
_________________________________________________________________


In [112]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [101]:
X_train_scaled.shape

(3832, 23)

In [102]:
df_clean_onehot.dtypes

age                               float64
avg_glucose_level                 float64
bmi                               float64
gender_Female                     float64
gender_Male                       float64
gender_Other                      float64
hypertension_0                    float64
hypertension_1                    float64
heart_disease_0                   float64
heart_disease_1                   float64
ever_married_No                   float64
ever_married_Yes                  float64
work_type_Govt_job                float64
work_type_Never_worked            float64
work_type_Private                 float64
work_type_Self-employed           float64
work_type_children                float64
Residence_type_Rural              float64
Residence_type_Urban              float64
smoking_status_Unknown            float64
smoking_status_formerly smoked    float64
smoking_status_never smoked       float64
smoking_status_smokes             float64
dtype: object

In [113]:
X_train_scaled = np.array([np.array(val) for val in X_train])
y_train_scaled = np.array([np.array(val) for val in y_train])

X_train = tf.cast(X_train_scaled , dtype=tf.float32)
y_train = tf.cast(y_train, dtype=tf.float32)

In [114]:
# Train the model

fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
120/120 [==============================] - 2s 3ms/step - loss: 23.4642 - accuracy: 0.1516
Epoch 2/100
120/120 [==============================] - 1s 5ms/step - loss: 1.8801 - accuracy: 0.8022
Epoch 3/100
120/120 [==============================] - 0s 4ms/step - loss: 0.4353 - accuracy: 0.9136
Epoch 4/100
120/120 [==============================] - 0s 3ms/step - loss: 0.3037 - accuracy: 0.9452
Epoch 5/100
120/120 [==============================] - 0s 3ms/step - loss: 0.2623 - accuracy: 0.9481
Epoch 6/100
120/120 [==============================] - 0s 3ms/step - loss: 0.2400 - accuracy: 0.9496
Epoch 7/100
120/120 [==============================] - 0s 3ms/step - loss: 0.2264 - accuracy: 0.9496
Epoch 8/100
120/120 [==============================] - 1s 6ms/step - loss: 0.2127 - accuracy: 0.9499
Epoch 9/100
120/120 [==============================] - 1s 7ms/step - loss: 0.2060 - accuracy: 0.9499
Epoch 10/100
120/120 [==============================] - 1s 7ms/step - loss: 0.1995 - accur

In [115]:
# hyperparameter tuning method
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Activation functions in list
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Initial
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=10), activation=activation, input_dim=len(X_train[0])))

    # Tune for optimum hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=10),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [120]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2)

In [121]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 508 Complete [00h 00m 39s]
val_accuracy: 0.9553990364074707

Best val_accuracy So Far: 0.9569640159606934
Total elapsed time: 01h 03m 49s


In [122]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 91,
 'num_layers': 1,
 'units_0': 81,
 'units_1': 11,
 'units_2': 21,
 'units_3': 21,
 'units_4': 1,
 'units_5': 21,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [123]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

40/40 - 0s - loss: 0.2356 - accuracy: 0.9570 - 284ms/epoch - 7ms/step
Loss: 0.2355785071849823, Accuracy: 0.9569640159606934


In [132]:
# Export our model to HDF5 file
filepath = r"...content/drive/MyDrive/stroke_data_nn_model.h5"
nn.save(filepath, save_format='h5')

In [136]:
# Set gdrive path
drive_path = '/content/drive/MyDrive/BootCamp/project_4/'

# Save the model to the specified path
best_model.save(drive_path + 'stroke_data_nn_model.h5')